# STAT 301 Assignment 1: Data and Question
### By William Cheng

In [2]:
# Run this cell before continuing.
library(tidyverse)
library(repr)
library(infer)
library(broom)
library(AER)
library(cowplot)
library(GGally)
census_data <- read_csv(file = "data/adult.data", col_names = FALSE, show_col_types = FALSE)
col_names = c("age", "workclass", "fnlwgt", "education", "education-num", "marital-status", "occupation", "relationship", "race", "sex", "capital-gain", "capital-loss", "hours-per-week", "native-country", "income")
colnames(census_data) <- col_names

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: car

Loading required package: carData


Attaching package: ‘car’


The following object is masked from ‘package:dplyr’:

    recode


The following object is masked from ‘package:purrr’:

    some


Loading required package: lmtest

Loading required package: zoo


Attaching package: ‘zoo’


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric


Loading required package: sandwich

Loading required package: survival


In [9]:
head(census_data)
nrow(census_data)
ncol(census_data)

age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K


[1] 32561

[1] 15

## Summary of Dataset

There are 32561 rows and 15 variables. The dataset was originally used to determine the target response variable, `income` from 14 features. Each rows contains the characteristic for `fnlwgt` number of individuals. Only individuals whose `age` > 16 were considered. The rows with columns containing the value "?" will be pruned in data cleaning. The following variables are as followed:
- `age`: numeric variable, the age of a person in years. Only individuals with `age` > 16 were considered.
- `workclass`: categorical variable, type chr, represents the type of employement, such as Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.
- `fnlwgt`: numeric variable, represent the final weight, meaning the number of people the census believes are represented by all the features in the row
- `education`: categorical variable, type chr, represents the highest education level obtained, such as Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
- `education-num`: numeric variable, the highest education level obtained in numeric form
- `marital-status`: categorical variable, type chr, represents the marital status for both civilian and Armed Forces marriages, such as Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
- `occupation`: categorical variable, type chr, represents the type of occupation the person has, such Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.
- `relationship`: categorical variable, type chr, represents the relations the person has to others, similar to `maritial-status`, such as Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.
- `race`: categorical variable, type chr, represents the race of the individual, such as White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.
- `sex`: categorical variable, type chr, represents the biological sex, such as Female, Male.
- `capital-gain`: numeric variable, the capital gain for the individual or profit made by selling assets.
- `capital-loss`: numeric variable, the capital loss for the individual or loss from selling assets.
- `hours-per-week`: numeric variable, the number of hours per week the individual works.
- `native-country`: categorical variable, the country where the individual is born, such as United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.
- `income`: categorical variable, type logical, whether the individual makes <=50K or > 50K

## Where Does the Dataset come from

The dataset comes from the UC Irvine Machine Learning Repository. It was collected from a US Census Bureau database in 1994 and extracted by Berry Becker. The intention was to predict whether a person makes <= 50K or > 50K from the given features.

The population of interest for the Census is the total population for all 50 state governments and 90299 local governments. Data was collected as a survey in the form of mail and internet, such that all US individuals should receive it. The Census data is then extracted to include adults only and income is masked to either <= 50K or > 50K. 

## Question

The question posed for this assignment is: 
Are `income` of <= 50K and > 50K, `race` and `sex` associated with the level of education of all American individuals?

Given that the `education-num` is a useful numeric indicator of education level, we could perform an additive multiple linear regression with the categorical/logical variables `income`, `sex`, and `race` and determine if the mean level of education are different. Then, we can determine if the mean level of education is statistically significant than the `income`.

We could create a response variable: $Y$ representing the mean education-num.

Then, we could create variables, $X_{female}$, $X_{>50K}$, and $X_{race}$ for 4 races other than the reference. We setting one value as a reference, so all $X$ is either 0 or 1.

This is an inferential question, such that the hypothesis tests below are describe as:

$H_0: \beta_i = 0$

$H_A: \beta_i \neq 0$

For $\beta$ of all variables $X$ and $\beta_0$ representing the intercept or the mean education-num of of the reference level.


Since we are dealing with the census, which is a big sample and similar to the population, we should take random samples from it. This is to preserve computational resources because the dataset contains high volume. As well, we can calculate use statistical inference such as confidence intervals and hypothesis tests to make inferences of the population.

## References

Becker, Barry and Kohavi, Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.

Bureau, U. C. (2023, May 17). How the data are collected. Census.gov. https://www.census.gov/programs-surveys/gov-finances/technical-documentation/methodology/how-the-data-are-collected.html#:~:text=Methods%3A%20The%20data%20collection%20for,state%20and%20type%20of%20government 